In [0]:
import os
import logging
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
from datetime import datetime, timedelta
import requests

In [0]:

def generate_presigned_url_eic(bucket_name, object_name, expiration):

    s3_client=boto3.client(
        "s3",
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_access_secret_key,
        region_name = "eu-west-1",
        config = Config(signature_version="s3v4")
    )

    try:
        response = s3_client.generate_presigned_url(
            "get_object",
            Params={
                "Bucket" : bucket_name,
                "Key" : object_name
            },
            ExpiresIn=expiration
        )
    except ClientError as e:
        logging.error(e)
        # print("AWS 자격 증명을 찾을 수 없습니다.")
        return None
    return response

In [0]:
# 이전 task 값 받아오기 
prev_task = "1218"
kst_date  = "20260123" # dbutils.jobs.taskValues.get(prev_task, "kst_date")

# access_key
aws_access_key = dbutils.secrets.get("admin","aws_common_access_key")
aws_access_secret_key = dbutils.secrets.get("admin","aws_common_secret_access_key") 

bucket_ls = ['s3-lge-he-inbound-eic-dev']
key = f"HEDS/{prev_task}/{kst_date}"


In [0]:
import boto3

file_list=[]
s3=boto3.resource(
    's3',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_access_secret_key
)

for bucket_nm in bucket_ls:
    bucket=s3.Bucket(bucket_nm)
    for obj in bucket.objects.filter(Prefix=key):
        if len(obj.key.split('/')[-1])>0:
            if obj.key.split('.')[-1] == 'csv':
                file_list.append((bucket_nm, obj.key))   
print(file_list)

In [0]:

if __name__ == "__main__":

    # 해당 prefix 하위의 파일을 모두 탐색한다.
    file_list=[]
    s3=boto3.resource(
        's3',
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_access_secret_key
    )
    for bucket_nm in bucket_ls:
        bucket=s3.Bucket(bucket_nm)
        for obj in bucket.objects.filter(Prefix=key):
            if len(obj.key.split('/')[-1])>0:
                file_list.append((bucket_nm, obj.key))   

    for bucket_name, object_name in file_list:
        eic_url = generate_presigned_url_eic(bucket_name, object_name, 259200)
    print(eic_url)
    # if eic_url:
    #     send_email(subject, content, mail_to, mail_from, mail_cc)
    # else:
    #     print('파일이 존재하지 않는 권역이 존재합니다.')


In [0]:
def send_email(subject, content, mail_to, mail_from, mail_cc):
    
    airflow_pw = dbutils.secrets.get("admin", "password_airflow_pw")

    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
    }

    json_data = {
        "conf" : {
            'mail_from' : mail_from,
            'mail_to' : mail_to,
            'mail_cc' : mail_cc,
            'subject' : subject,
            'content_str' : content,
            'content_type' : 'plain',
        }
    }

    response = requests.post(
        'https://airflow-kic.hedataplfm.com/api/v1/dags/smtp_call_str/dagRuns',
        headers=headers,
        json=json_data,
        auth=('admin', airflow_pw)
    )

    if response.status_code != 200:
        print(response.text)

In [0]:
subject = f'[HEDS-1043]DRM 노말로그 주기적 추출 요청 {kst_date}'
content = f"""
    안녕하십니까? msdp입니다.

    DRM 결과 파일 전달 드립니다.
    EIC :
    {eic_url}

    다운로드 가능 기간은 현시점으로부터 72시간 내에만 가능합니다.

    감사합니다.
"""
mail_to = 'eunmi1.ko@lge.com'
mail_from = 'eunmi1.ko@lge.com'
mail_cc = 'eunmi1.ko@lge.com'
